In [1]:
# Import required libraries and dependencies
import pandas as pd
import hvplot.pandas
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

In [2]:
# Load the data into a Pandas DataFrame
df_market_data = pd.read_csv(
    "Resources/crypto_market_data.csv",
    index_col="coin_id")
# Display sample data
df_market_data.head(10)

,price_change_percentage_24h,price_change_percentage_7d,price_change_percentage_14d,price_change_percentage_30d,price_change_percentage_60d,price_change_percentage_200d,price_change_percentage_1y
coin_id,,,,,,,
bitcoin,1.08388,7.60278,6.57509,7.67258,-3.25185,83.51840,37.51761
ethereum,0.22392,10.38134,4.80849,0.13169,-12.88890,186.77418,101.96023
tether,-0.21173,0.04935,0.00640,-0.04237,0.28037,-0.00542,0.01954
ripple,-0.37819,-0.60926,2.24984,0.23455,-17.55245,39.53888,-16.60193
bitcoin-cash,2.90585,17.09717,14.75334,15.74903,-13.71793,21.66042,14.49384
binancecoin,2.10423,12.85511,6.80688,0.05865,36.33486,155.61937,69.69195
chainlink,-0.23935,20.69459,9.30098,-11.21747,-43.69522,403.22917,325.13186
cardano,0.00322,13.99302,5.55476,10.10553,-22.84776,264.51418,156.09756
litecoin,-0.06341,6.60221,7.28931,1.21662,-17.23960,27.49919,-12.66408


In [3]:
# Generate summary statistics
df_market_data.describe()

,price_change_percentage_24h,price_change_percentage_7d,price_change_percentage_14d,price_change_percentage_30d,price_change_percentage_60d,price_change_percentage_200d,price_change_percentage_1y
count,41.000000,41.000000,41.000000,41.000000,41.000000,41.000000,41.000000
mean,-0.269686,4.497147,0.185787,1.545693,-0.094119,236.537432,347.667956
std,2.694793,6.375218,8.376939,26.344218,47.365803,435.225304,1247.842884
min,-13.527860,-6.094560,-18.158900,-34.705480,-44.822480,-0.392100,-17.567530
25%,-0.608970,0.047260,-5.026620,-10.438470,-25.907990,21.660420,0.406170
50%,-0.063410,3.296410,0.109740,-0.042370,-7.544550,83.905200,69.691950
75%,0.612090,7.602780,5.510740,4.578130,0.657260,216.177610,168.372510
max,4.840330,20.694590,24.239190,140.795700,223.064370,2227.927820,7852.089700


In [4]:
# Plot your data to see what's in your DataFrame
df_market_data.hvplot.line(
    width=800,
    height=400,
    rot=90
)

:NdOverlay   [Variable]
   :Curve   [coin_id]   (value)

---

### Prepare the Data

In [5]:
# Use the `StandardScaler()` module from scikit-learn to normalize the data from the CSV file
scaler = StandardScaler()
df_market_data_scaled = scaler.fit_transform(df_market_data[["price_change_percentage_24h", "price_change_percentage_7d", "price_change_percentage_14d"]])

In [6]:
# Create a DataFrame with the scaled data
# Copy the crypto names from the original DataFrame
# Set the coin_id column as index
df_market_data_scaled = pd.DataFrame(df_market_data_scaled, columns=["price_change_percentage_24h", "price_change_percentage_7d", "price_change_percentage_14d"], index=df_market_data.index)
# Display the scaled DataFrame
df_market_data_scaled.head()

,price_change_percentage_24h,price_change_percentage_7d,price_change_percentage_14d
coin_id,,,
bitcoin,0.508529,0.493193,0.772200
ethereum,0.185446,0.934445,0.558692
tether,0.021774,-0.706337,-0.021680
ripple,-0.040764,-0.810928,0.249458
bitcoin-cash,1.193036,2.000959,1.760610


---

### Find the Best Value for k Using the Original Data.

In [7]:
# Create a list with the number of k-values from 1 to 11
k_values = range(1, 12)

In [8]:
# Create an empty list to store the inertia values
# Create a for loop to compute the inertia with each possible value of k
# Inside the loop:
# 1. Create a KMeans model using the loop counter for the n_clusters
# 2. Fit the model to the data using `df_market_data_scaled`
# 3. Append the model.inertia_ to the inertia list
inertia_values = []

for k in k_values:
    kmeans = KMeans(n_clusters=k, random_state=0)
    kmeans.fit(df_market_data_scaled)
    inertia_values.append(kmeans.inertia_)

c:\Users\Cloud\AN3\envs\dev\lib\site-packages\joblib\externals\loky\backend\context.py:136: UserWarning: Could not find the number of physical cores for the following reason:
[WinError 2] The system cannot find the file specified
Returning the number of logical cores instead. You can silence this warning by setting LOKY_MAX_CPU_COUNT to the number of cores you want to use.
  warnings.warn(
  File "c:\Users\Cloud\AN3\envs\dev\lib\site-packages\joblib\externals\loky\backend\context.py", line 257, in _count_physical_cores
    cpu_info = subprocess.run(
  File "c:\Users\Cloud\AN3\envs\dev\lib\subprocess.py", line 503, in run
    with Popen(*popenargs, **kwargs) as process:
  File "c:\Users\Cloud\AN3\envs\dev\lib\subprocess.py", line 971, in __init__
    self._execute_child(args, executable, preexec_fn, close_fds,
  File "c:\Users\Cloud\AN3\envs\dev\lib\subprocess.py", line 1456, in _execute_child
    hp, ht, pid, tid = _winapi.CreateProcess(executable, args,
c:\Users\Cloud\AN3\envs\dev\lib

In [9]:
# Create a dictionary with the data to plot the Elbow curve
elbow_data = {"k" : k_values, "inertia" : inertia_values}
# Create a DataFrame with the data to plot the Elbow curve
elbow_df = pd.DataFrame(elbow_data)

In [10]:
# Plot a line chart with all the inertia values computed with
# the different values of k to visually identify the optimal value for k.
elbow_df.hvplot.line(x="k", y="inertia", title="Elbow Curve")

:Curve   [k]   (inertia)

#### Answer the following question: 

**Question:** What is the best value for `k`?

**Answer:** Based on the elbow curves generated, the best value for `k` is 4.

---

### Cluster Cryptocurrencies with K-means Using the Original Scaled DataFrame

In [11]:
# Initialise the K-Means model using the best value for k
best_k = 4
kmeans = KMeans(n_clusters=best_k, random_state=0)

In [12]:
# Fit the K-Means model using the scaled DataFrame
kmeans.fit(df_market_data_scaled)

c:\Users\Cloud\AN3\envs\dev\lib\site-packages\sklearn\cluster\_kmeans.py:1446: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=1.
  warnings.warn(


KMeans(n_clusters=4, random_state=0)

In [13]:
# Predict the clusters to group the cryptocurrencies using the scaled DataFrame
clusters = kmeans.predict(df_market_data_scaled)
# Print the resulting array of cluster values.
clusters

array([3, 3, 0, 0, 1, 1, 1, 1, 3, 0, 0, 0, 0, 1, 3, 3, 0, 0, 1, 0, 0, 3,
       0, 0, 0, 0, 0, 0, 3, 0, 3, 0, 2, 3, 0, 0, 3, 0, 3, 0, 0])

In [14]:
# Create a copy of the scaled DataFrame
df_market_data_scaled_copy = df_market_data_scaled.copy()

In [15]:
# Add a new column to the copy of the scaled DataFrame with the predicted clusters
df_market_data_scaled_copy["cluster"] = clusters
# Display the copy of the scaled DataFrame
df_market_data_scaled_copy.head()

,price_change_percentage_24h,price_change_percentage_7d,price_change_percentage_14d,cluster
coin_id,,,,
bitcoin,0.508529,0.493193,0.772200,3
ethereum,0.185446,0.934445,0.558692,3
tether,0.021774,-0.706337,-0.021680,0
ripple,-0.040764,-0.810928,0.249458,0
bitcoin-cash,1.193036,2.000959,1.760610,1


In [16]:
# Create a scatter plot using hvPlot by setting
# `x="price_change_percentage_24h"` and `y="price_change_percentage_7d"`.
# Colour the graph points with the labels found using K-Means and
# add the crypto name in the `hover_cols` parameter to identify
# the cryptocurrency represented by each data point.
df_market_data_scaled_copy.hvplot.scatter(
    x="price_change_percentage_24h",
    y="price_change_percentage_7d",
    by="cluster",
    hover_cols=["coin_id"]
)

:NdOverlay   [cluster]
   :Scatter   [price_change_percentage_24h]   (price_change_percentage_7d,coin_id)

---

### Optimise Clusters with Principal Component Analysis.

In [17]:
# Create a PCA model instance and set `n_components=3`.
pca = PCA(n_components=3)

In [18]:
# Use the PCA model with `fit_transform` to reduce the original scaled Dataframe
# down to three principal components.
df_market_data_pca = pca.fit_transform(df_market_data_scaled)
df_market_data_pca_df = pd.DataFrame(df_market_data_pca, columns=["PC1", "PC2", "PC3"], index=df_market_data.index)
# View the scaled PCA data
df_market_data_pca_df.head()

,PC1,PC2,PC3
coin_id,,,
bitcoin,-1.030769,-0.110739,0.152931
ethereum,-1.027774,0.324361,-0.241214
tether,0.444212,-0.319032,0.448052
ripple,0.356802,-0.259455,0.725858
bitcoin-cash,-2.909217,0.084670,-0.237078


In [19]:
# Retrieve the explained variance to determine how much information
# can be attributed to each principal component.
explained_variance = pca.explained_variance_ratio_
total_explained_variance = explained_variance.sum()
total_explained_variance

1.0000000000000002

#### Answer the following question: 

**Question:** What is the total explained variance of the three principal components?

**Answer:** The total explained variance of the three principal components is 1.0000000000000002, which is approximately 100%.

In [20]:
# Create a new DataFrame with the PCA data.
# Copy the crypto names from the original scaled DataFrame
# Set the coin_id column as index
# Display the scaled PCA DataFrame
df_market_data_pca_df = pd.DataFrame(df_market_data_pca, columns=["PC1", "PC2", "PC3"], index=df_market_data.index)
df_market_data_pca_df.head()

,PC1,PC2,PC3
coin_id,,,
bitcoin,-1.030769,-0.110739,0.152931
ethereum,-1.027774,0.324361,-0.241214
tether,0.444212,-0.319032,0.448052
ripple,0.356802,-0.259455,0.725858
bitcoin-cash,-2.909217,0.084670,-0.237078


---

### Find the Best Value for k Using the Scaled PCA DataFrame

In [21]:
# Create a list with the number of k-values from 1 to 11
k_values_pca = range(1, 12)

In [22]:
# Create an empty list to store the inertia values
# Create a for loop to compute the inertia with each possible value of k
# Inside the loop:
# 1. Create a KMeans model using the loop counter for the n_clusters
# 2. Fit the model to the data using `df_market_data_pca`
# 3. Append the model.inertia_ to the inertia list
inertia_values_pca = []

for k in k_values_pca:
    kmeans_pca = KMeans(n_clusters=k, random_state=0)
    kmeans_pca.fit(df_market_data_pca_df)
    inertia_values_pca.append(kmeans_pca.inertia_)

c:\Users\Cloud\AN3\envs\dev\lib\site-packages\sklearn\cluster\_kmeans.py:1446: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=1.
  warnings.warn(
c:\Users\Cloud\AN3\envs\dev\lib\site-packages\sklearn\cluster\_kmeans.py:1446: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=1.
  warnings.warn(
c:\Users\Cloud\AN3\envs\dev\lib\site-packages\sklearn\cluster\_kmeans.py:1446: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=1.
  warnings.warn(
c:\Users\Cloud\AN3\envs\dev\lib\site-packages\sklearn\cluster\_kmeans.py:1446: UserWarning: KMeans is known to have a memory le

In [23]:
# Create a dictionary with the data to plot the Elbow curve
elbow_data_pca = {"k": k_values_pca, "inertia": inertia_values_pca}
# Create a DataFrame with the data to plot the Elbow curve
elbow_pca_df = pd.DataFrame(elbow_data_pca)

In [24]:
# Plot a line chart with all the inertia values computed with
# the different values of k to visually identify the optimal value for k.
elbow_pca_df.hvplot.line(x="k", y="inertia", title="Elbow Curve (PCA Data)")

:Curve   [k]   (inertia)

#### Answer the following questions: 

* **Question:** What is the best value for `k` when using the PCA data?

  * **Answer:** The best value for `k` when using the PCA data is 3.


* **Question:** Does it differ from the best k value found using the original data?

  * **Answer:** Yes, it differs. The best value for `k` using the original data is 4.

### Cluster Cryptocurrencies with K-means Using the Scaled PCA DataFrame

In [25]:
# Initialise the K-Means model using the best value for k
best_k_pca = 3
kmeans_pca = KMeans(n_clusters=best_k_pca, random_state=0)

In [26]:
# Fit the K-Means model using the scaled PCA DataFrame
kmeans_pca.fit(df_market_data_pca_df)

c:\Users\Cloud\AN3\envs\dev\lib\site-packages\sklearn\cluster\_kmeans.py:1446: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=1.
  warnings.warn(


KMeans(n_clusters=3, random_state=0)

In [27]:
# Predict the clusters to group the cryptocurrencies using the scaled PCA DataFrame
clusters_pca = kmeans_pca.predict(df_market_data_pca_df)
# Print the resulting array of cluster values.
clusters_pca

array([1, 1, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 1,
       0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 2, 0, 0, 0, 1, 0, 0, 0, 0])

In [28]:
# Create a copy of the scaled PCA DataFrame
df_market_data_pca_copy = df_market_data_pca_df.copy()
# Add a new column to the copy of the PCA DataFrame with the predicted clusters
df_market_data_pca_copy["cluster"] = clusters_pca
# Display the copy of the scaled PCA DataFrame
df_market_data_pca_copy.head()


,PC1,PC2,PC3,cluster
coin_id,,,,
bitcoin,-1.030769,-0.110739,0.152931,1
ethereum,-1.027774,0.324361,-0.241214,1
tether,0.444212,-0.319032,0.448052,0
ripple,0.356802,-0.259455,0.725858,0
bitcoin-cash,-2.909217,0.084670,-0.237078,1


In [29]:
# Create a scatter plot using hvPlot by setting
# `x="PC1"` and `y="PC2"`.
# Colour the graph points with the labels found using K-Means and
# add the crypto name in the `hover_cols` parameter to identify
# the cryptocurrency represented by each data point.
df_market_data_pca_copy.hvplot.scatter(
    x="PC1",
    y="PC2",
    by="cluster",
    hover_cols=["coin_id"]
)

:NdOverlay   [cluster]
   :Scatter   [PC1]   (PC2,coin_id)

### Visualise and Compare the Results

In this section, you will visually analyse the cluster analysis results by contrasting the outcome with and without using the optimisation techniques.

In [30]:
# Composite plot to contrast the Elbow curves
(elbow_df.hvplot.line(x="k", y="inertia", title="Elbow Curve (Original)") +
elbow_pca_df.hvplot.line(x="k", y="inertia", title="Elbow Curve (PCA Data)")).cols(1)

:Layout
   .Curve.I  :Curve   [k]   (inertia)
   .Curve.II :Curve   [k]   (inertia)

In [31]:
# Composite plot to contrast the clusters
df_market_data_scaled_copy.hvplot.scatter(
    x="price_change_percentage_24h",
    y="price_change_percentage_7d")

:Scatter   [price_change_percentage_24h]   (price_change_percentage_7d)

#### Answer the following question: 

  * **Question:** After visually analysing the cluster analysis results, what is the impact of using fewer features to cluster the data using K-Means?

  * **Answer:** Using fewer features with PCA simplifies the data and makes computation faster, while still preserving the main clusters. The clustering results remain consistent, but with a slight loss of detail. The reduced feature set makes it easier to visualize and interpret the data.